In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode
import os
import json
import time
# TOPOLOGIES
from network_topologies.pseudogeant import Pseudogeant
from network_topologies.geant import Geant
from network_topologies.topology import Topology
from network_topologies.topology_preprocessing import preprocess_metadata
# ROUTING ALGORITHMS
from routing_algorithms.dijkstra import dijkstra
from routing_algorithms.dijkstra import calculate_path
from routing_algorithms.dijkstra import set_spt
from routing_algorithms.ear import ear
from routing_algorithms.mora import *
# NETWORK OBJECTS
from utils.network_objects import *
# PA SERVICES
from service_flows.traffic_generator import TrafficGenerator
from itertools import permutations
import random
import numpy as np
from deap import algorithms, base, creator, tools, algorithms
import collections
import matplotlib.pyplot as plt
import import_ipynb

In [6]:
topo = Geant()
topo.save_topology_info()

# INSTANTIATE TRAFFIC GENERATOR
set_spt(topo)
tg = TrafficGenerator(interval=300, topology=topo)
time.sleep(1)

# Get flows to be routed
service_flows = tg.flows
# get flow from node1 to node2 of type 'premium'


In [33]:
def optimize_route(topology, flow_dic):
    flow_obj = Flow(flow_dic)
    # number of weights in the tuple -> number of objective functions
    creator.create("FitnessMultiObj", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0,)) 
    creator.create("Individual", list, fitness=creator.FitnessMultiObj)
    toolbox = base.Toolbox()
    toolbox.register("individual", generate_individual, creator.Individual, flow_obj.starting_node, flow_obj.ending_node, topology)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", crossover_one_point, topology=topology, ind_class=creator.Individual, toolbox=toolbox)
    toolbox.register("select", tools.selNSGA2)
    toolbox.register("mutate", mutate_path, topology=topology, indi_class=creator.Individual)
    
    # FIXME
    evaluate_individual = get_evaluate_individual(topology, flow_dic)
    toolbox.register("evaluate", evaluate_individual)
    evaluate_SLA = get_evaluate_SLA(flow_obj.SLA, topology, evaluate_individual)
    penalty = get_penalty(flow_obj.SLA, topology, evaluate_individual) 
    toolbox.decorate("evaluate", tools.DeltaPenality(evaluate_SLA, 20.0, penalty))

    pop = toolbox.population(n=50)
    #FIXME
    hof = tools.ParetoFront(similar = compare_individuals) #  similar=np.array_equal
    stats = tools.Statistics(lambda ind: ind.fitness.values)

    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof)


    return hof

In [28]:
service_flows

{'ATESassured': {'_id': 'ATESassured',
  'bandwidth': 2659.9,
  'jitter': 0,
  'latency': 400,
  'loss': 0,
  'node1': 'AT',
  'node2': 'ES'},
 'ATESbesteffort': {'_id': 'ATESbesteffort',
  'bandwidth': 674.9,
  'jitter': 0,
  'latency': 300000,
  'loss': 0,
  'node1': 'AT',
  'node2': 'ES'},
 'ATESpremium': {'_id': 'ATESpremium',
  'bandwidth': 635.2,
  'jitter': 0,
  'latency': 150,
  'loss': 0,
  'node1': 'AT',
  'node2': 'ES'},
 'ATFRassured': {'_id': 'ATFRassured',
  'bandwidth': 2274.65,
  'jitter': 0,
  'latency': 400,
  'loss': 0,
  'node1': 'AT',
  'node2': 'FR'},
 'ATFRbesteffort': {'_id': 'ATFRbesteffort',
  'bandwidth': 577.15,
  'jitter': 0,
  'latency': 300000,
  'loss': 0,
  'node1': 'AT',
  'node2': 'FR'},
 'ATFRpremium': {'_id': 'ATFRpremium',
  'bandwidth': 543.2,
  'jitter': 0,
  'latency': 150,
  'loss': 0,
  'node1': 'AT',
  'node2': 'FR'},
 'BGSKassured': {'_id': 'BGSKassured',
  'bandwidth': 270.68,
  'jitter': 0,
  'latency': 400,
  'loss': 0,
  'node1': 'BG',
 

In [ ]:
for f in service_flows.keys():
    result = optimize_route(topo, service_flows[f])
    path = result[0]
    topo.apply_service_on_network(flow_dic, path)

gen	nevals	avg                                                  	std                                              	min                                                  	max                                                  
0  	50    	[ 11.6         20.562      247.49792518  16.        ]	[ 4.45421149 11.56032681 77.28668203  0.        ]	[  6.          11.8        134.15997916  16.        ]	[ 24.          55.9        447.80132859  16.        ]
1  	42    	[ 11.6         20.668      248.22279942  16.        ]	[ 4.41814441 11.98171006 75.88628449  0.        ]	[  7.          11.8        139.08667853  16.        ]	[ 24.          54.8        437.12124346  16.        ]
2  	36    	[ 11.56        20.428      247.61823968  16.        ]	[ 4.47285144 12.1875845  75.64033607  0.        ]	[  7.          11.8        139.08667853  16.        ]	[ 24.          54.8        437.12124346  16.        ]
3  	32    	[ 11.56        20.29       248.28465361  16.        ]	[ 4.47732063 12.21376682 75.11064482  0.   

In [39]:
topo.remove_service_from_network(f, path)

In [ ]:
indi = []
for i in range(5000):
    indi.append(generate_individual(list, flow.starting_node, flow.ending_node, topo))

In [ ]:
for ind in indi:
    p = mutate_path(ind, topo, list)
    print(p[0])
    assert not topo.has_loops(p[0])
    assert topo.is_valid(p[0])

In [14]:
(-1.0,) * 3

(-1.0, -1.0, -1.0)